In [1]:
'''
    Used code of https://github.com/random-forests/tutorials
'''

'\n    Used code of https://github.com/random-forests/tutorials\n'

In [2]:
import numpy as np

raw = np.genfromtxt('hour.csv', delimiter = ',')
print(raw)

# read raw data into target/feature by discrete form
# target
cnt_median = np.median(raw[1:, -1:])
target = []
for i in raw[1:, -1:]:
    if i < cnt_median:
        target.append('Grow')
    else:
        target.append('Drop')
target = np.array(target)
print(target)
# features considered
feature_name = ['Season', 'Period of day', 'Holiday', 'Weather', 'Temperature', 'Humidity', 'Wind speed'] 
# cal median (better than mean)
temp_median = np.median(raw[1:, 10])
hum_median = np.median(raw[1:, 12])
wind_median = np.median(raw[1:, 13])
# Generate feature data 
feature = [[], [], [], [], [], [], []]
# A little mistake, use transpose below
for i in raw[1:]:
    # season
    if i[2] == 1:
        feature[0].append('Spring')
    elif i[2] == 2:
        feature[0].append('Summer')
    elif i[2] == 3:
        feature[0].append('Autumn')
    else:
        feature[0].append('Winter')
    # hour
    if i[5] < 6:
        feature[1].append('Early morning')
    elif i[5] < 12:
        feature[1].append('Morning')
    elif i[5] < 18:
        feature[1].append('Afternoon')
    else:
        feature[1].append('Evening')
    # holiday
    if i[6] == 1:
        feature[2].append('Yes')
    else:
        feature[2].append('No')
    '''
    # weekday
    cause the ugly tree
    if i[7] == 0:
        feature[3].append('Sun')
    elif i[7] == 1:
        feature[3].append('Mon')
    elif i[7] == 2:
        feature[3].append('Tue')
    elif i[7] == 3:
        feature[3].append('Wed') 
    elif i[7] == 4:
        feature[3].append('Thu')
    elif i[7] == 5:
        feature[3].append('Fri') 
    else:
        feature[3].append('Sat')
    '''
    # weather
    if i[9] == 1:
        feature[3].append('Best')
    elif i[9] == 2:
        feature[3].append('Good')
    elif i[9] == 3:
        feature[3].append('Bad')
    else:
        feature[3].append('Very bad')
    # temp
    if i[10] > temp_median:
        feature[4].append(f'>{int(temp_median*41)}')
    else:
        feature[4].append(f'<{int(temp_median*41)}')
    # humidity
    if i[12] > hum_median:
        feature[5].append(f'>{int(hum_median*100)}')
    else:
        feature[5].append(f'<{int(hum_median*100)}')
    # wind speed
    if i[13] > wind_median:
        feature[6].append(f'>{int(wind_median*67)}')
    else:
        feature[6].append(f'<{int(wind_median*67)}')
feature = np.array(feature).T
print(feature)
target = target.reshape(len(target), 1)
train = np.concatenate((feature, target), axis=1)
header = ['Season', 'Period of day', 'Holiday', 'Weather', 'Temperature', 'Humidity', 'Wind speed']

[[       nan        nan        nan ...        nan        nan        nan]
 [1.0000e+00        nan 1.0000e+00 ... 3.0000e+00 1.3000e+01 1.6000e+01]
 [2.0000e+00        nan 1.0000e+00 ... 8.0000e+00 3.2000e+01 4.0000e+01]
 ...
 [1.7377e+04        nan 1.0000e+00 ... 7.0000e+00 8.3000e+01 9.0000e+01]
 [1.7378e+04        nan 1.0000e+00 ... 1.3000e+01 4.8000e+01 6.1000e+01]
 [1.7379e+04        nan 1.0000e+00 ... 1.2000e+01 3.7000e+01 4.9000e+01]]
['Grow' 'Grow' 'Grow' ... 'Grow' 'Grow' 'Grow']
[['Spring' 'Early morning' 'No' ... '<20' '>63' '<12']
 ['Spring' 'Early morning' 'No' ... '<20' '>63' '<12']
 ['Spring' 'Early morning' 'No' ... '<20' '>63' '<12']
 ...
 ['Spring' 'Evening' 'No' ... '<20' '<63' '<12']
 ['Spring' 'Evening' 'No' ... '<20' '<63' '<12']
 ['Spring' 'Evening' 'No' ... '<20' '>63' '<12']]


In [3]:
def class_counts(rows):
    counts = {}
    for row in rows:
        label = row[-1]
        if label not in counts:
            counts[label] = 0
        counts[label] += 1
    return counts

In [4]:
def is_numeric(value):
    return isinstance(value, int) or isinstance(value, float)

In [5]:
class Question:
    def __init__(self, column, value):
        self.column = column
        self.value = value

    def match(self, example):
        val = example[self.column]
        if is_numeric(val):
            return val >= self.value
        else:
            return val == self.value

    def __repr__(self):
        condition = "=="
        if is_numeric(self.value):
            condition = ">="
        return "Is %s %s %s?" % (
            header[self.column], condition, str(self.value))

In [6]:
def partition(rows, question):
    true_rows, false_rows = [], []
    for row in rows:
        if question.match(row):
            true_rows.append(row)
        else:
            false_rows.append(row)
    return true_rows, false_rows

In [7]:
def gini(rows):
    counts = class_counts(rows)
    impurity = 1
    for lbl in counts:
        prob_of_lbl = counts[lbl] / float(len(rows))
        impurity -= prob_of_lbl**2
    return impurity

In [8]:
def info_gain(left, right, current_uncertainty):
    p = float(len(left)) / (len(left) + len(right))
    return current_uncertainty - p * gini(left) - (1 - p) * gini(right)

In [9]:
def find_best_split(rows):
    best_gain = 0  # keep track of the best information gain
    best_question = None  # keep train of the feature / value that produced it
    current_uncertainty = gini(rows)
    n_features = len(rows[0]) - 1  # number of columns

    for col in range(n_features):  # for each feature

        values = set([row[col] for row in rows])  # unique values in the column

        for val in values:  # for each value

            question = Question(col, val)

            # try splitting the dataset
            true_rows, false_rows = partition(rows, question)

            # Skip this split if it doesn't divide the
            # dataset.
            if len(true_rows) == 0 or len(false_rows) == 0:
                continue

            # Calculate the information gain from this split
            gain = info_gain(true_rows, false_rows, current_uncertainty)

            # You actually can use '>' instead of '>=' here
            # but I wanted the tree to look a certain way for our
            # toy dataset.
            if gain >= best_gain:
                best_gain, best_question = gain, question

    return best_gain, best_question

In [10]:
class Leaf:
    def __init__(self, rows):
        self.predictions = class_counts(rows)

In [11]:
class Decision_Node:
    def __init__(self,
                 question,
                 true_branch,
                 false_branch):
        self.question = question
        self.true_branch = true_branch
        self.false_branch = false_branch

In [12]:
def build_tree(rows):
    gain, question = find_best_split(rows)
    if gain == 0:
        return Leaf(rows)
    true_rows, false_rows = partition(rows, question)
    true_branch = build_tree(true_rows)
    false_branch = build_tree(false_rows)
    return Decision_Node(question, true_branch, false_branch)

In [13]:
def print_tree(node, spacing=""):
    # Base case: we've reached a leaf
    if isinstance(node, Leaf):
        print (spacing + "Predict", node.predictions)
        return
    print (spacing + str(node.question))
    print (spacing + '--> True:')
    print_tree(node.true_branch, spacing + "  ")
    print (spacing + '--> False:')
    print_tree(node.false_branch, spacing + "  ")

In [14]:
my_tree = build_tree(train)

In [15]:
print_tree(my_tree)

Is Period of day == Early morning?
--> True:
  Is Temperature == <20?
  --> True:
    Is Season == Winter?
    --> True:
      Is Humidity == <63?
      --> True:
        Is Wind speed == <12?
        --> True:
          Is Weather == Best?
          --> True:
            Is Holiday == No?
            --> True:
              Predict {'Grow': 72, 'Drop': 4}
            --> False:
              Predict {'Grow': 4}
          --> False:
            Predict {'Grow': 21}
        --> False:
          Is Weather == Good?
          --> True:
            Predict {'Grow': 31, 'Drop': 1}
          --> False:
            Predict {'Grow': 86}
      --> False:
        Is Wind speed == <12?
        --> True:
          Is Weather == Bad?
          --> True:
            Predict {'Grow': 26}
          --> False:
            Is Holiday == No?
            --> True:
              Is Weather == Good?
              --> True:
                Predict {'Grow': 132, 'Drop': 1}
              --> False:
           